# Ratterdam Beltway Population Decoding Analysis
## Non-stationary analysis and decoding approaches
## Decoding each alley in turn approaches

In [1]:
import sklearn as skl
from sklearn import svm, preprocessing, metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn import neighbors
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
from scipy import interp
from scipy.integrate import simps
from scipy.ndimage import center_of_mass
import numpy as np, random, json, pickle, datetime, copy, socket, os, sys, scipy
from scipy.stats import sem
import matplotlib.colors as colors
from importlib import reload

import utility_fx as util
import ratterdam_ParseBehavior as Parse
import ratterdam_CoreDataStructures as Core
import ratterdam_PermutationTests as Perm
import ratterdam_Defaults as Def
import ratterdam_DataFiltering as Filt

In [2]:
%matplotlib qt5
%qtconsole --style native

## Helper Functions and Parameters

In [3]:
def checkRM(ratemap):
    """Utility function to take a 1-d linear ratemap
    and see if it is valid.
    May 2019: it's not empty ie. there's data
    and the nanmax of that data exceeds firing rate
    thresh defined locally"""
    if type(ratemap) == np.ndarray and np.nanmax(ratemap) >= frThresh:
        return True
    else:
        return False

In [4]:
def createVisitSummaryFeatures(unit, alley, visit, features):
    """
    For a given pass for a given unit summarize the 1d ratemap into a simpler,
    explicit vector of attributes. Which attributes to use are given by
    the 'features' list.
    """
    feats = np.empty((0))
    rm = unit.alleys[alley][visit]['ratemap1d']
    # i dont know of a better way of doing this other than to just check param name in list and add it if present
    if 'rm' in features:
        feats = np.append(feats, rm)
    if 'time' in features:
        feats = np.append(feats, visit)
    if 'max95' in features:
        maximum = np.nanpercentile(rm, 95)
        feats = np.append(feats, maximum)
    if 'locmax95' in features:
        locmax = np.searchsorted(np.sort(rm), np.percentile(rm, 95))
        feats = np.append(feats, locmax)
    if 'mean' in features:
        mean = np.nanmean(rm)
        feats = np.append(feats, mean)
    if 'auc' in features:
        auc = simps(rm)
        feats = np.append(feats, auc)
    if 'avgdrds' in features:
        avgdrds = np.mean(np.abs(np.diff(rm))) # avg dr/ds change in rate / change in pos. 
        feats = np.append(feats, avgdrds)
    if 'maxdrds' in features:
        maxdrds = np.percentile(np.abs(np.diff(rm)), 95)
        feats = np.append(feats, maxdrds)
    if 'com' in features:
        try:
            com = center_of_mass(rm)[0]
            feats = np.append(feats, com)
        except:
            com = int(round(Def.singleAlleyBins[1]-1)/2)
            feats = np.append(feats, com)
    if 'comval' in features:
        try:
            comval = rm[int(np.round(com))]
            feats  = np.append(feats, comval)
        except:
            comval = np.nanpercentile(rm, 95)
            feats  = np.append(feats, comval)
    if 'boundMaxs' in features:
        # think there may be something going on at entrace/exit to alley so get the max val 
        # within each trisector of alley. NB real intersection ends with alleybounds_manuallyshifted2
        # gives a 3:6:3 ratio of approach L:alley:approach/exit R but I want to squeeze in the bounds to give
        # more space to the flanks (4:4:4 ratio) to capture whats happening at boundary itself as well.
        max1,max2,max3 = np.nanmax(rm[:4]), np.nanmax(rm[4:8]), np.nanmax(rm[9:]) # assumes 12bin rm. make generalized later
        feats = np.append(feats,(max1,max2,max3))
    return feats

In [5]:
def setupAlleyData(alley, representationFx, features, bootstrap):

    """
    Function that creates a population feature matrix for an alley
    along with a corresponding label vector
    
    Feature matrix X is (t,n*f) where t is number trials, n is number cells, f is number features / cell
    Label vector Y is (t,1) where each entry is the stimulus label for that trial
    Features f are found for each cell via representrationFx which is the generic name
    for any fx that takes (Unit, visit) as args and returns a (f,1) array of the features
    for that cell for that visit. What the features are is determined by user in code of the rep fx.
    Matrix X is transformed via scikitlearn's preprocessing.StandardScalar().fit_transform() fx to have 0mean,unit var
    
    Returns X (t,n*f) array feature matrix trials by pop feature vector
            Y (t,1) array of trial labels

    """
    X = []
    Y = np.empty((0))
    
    if bootstrap == True:
        bootnames = np.random.choice(list(population.keys()), len(population.keys()), replace=True)

    for visit in range(len(population[list(population.keys())[0]].alleys[alley])):

        stim = population[list(population.keys())[0]].alleys[alley][visit]['metadata']['stimulus']
        dataRow = np.empty((0))
        
        for unitname, Unit in population.items():
            if bootstrap == True:
                if unitname in bootnames:
                    valid = Filt.checkOverallAlleyActivity(Unit, alley)
                    if valid:
                        rm = representationFx(Unit, alley, visit, features)
                        dataRow = np.hstack((dataRow, rm))
                        
            elif bootstrap == False:
                valid = Filt.checkOverallAlleyActivity(Unit, alley)
                if valid:
                    rm = representationFx(Unit, alley, visit, features)
                    dataRow = np.hstack((dataRow, rm))
                    

        Y = np.append(Y, stim)
        X.append(dataRow)
    X = np.asarray(X)
    X[np.where(~np.isfinite(X))] = 0
    X = preprocessing.StandardScaler().fit_transform(X)

    return X, Y

In [6]:
frThresh = 0 #measured in Hz. Pick something close to 0, or 0 itself. 
beltwayAlleys = [16, 17, 3, 1, 5, 7, 8, 10, 11] # beltway alley IDs in terms of their full track, 17-alley ID
nbins = Def.singleAlleyBins[0]-1

### Load Data

In [7]:
rat = 'R808'
expCode = "BRD6"
datafile = f"E:\\Ratterdam\\{rat}\\{rat}{expCode}\\"

alleyTracking, alleyVisits,  txtVisits, p_sess, ts_sess = Parse.getDaysBehavioralData(datafile, expCode)
population = {}
for subdir, dirs, fs in os.walk(datafile):
    for f in fs:
        if 'cl-maze1' in f and 'OLD' not in f and 'Undefined' not in f:
            clustname = subdir[subdir.index("TT"):] + "\\" + f
            unit = Core.UnitData(clustname, datafile, expCode, Def.alleyBounds, alleyVisits, txtVisits, p_sess, ts_sess)
            unit.loadData_raw()
            rm = util.makeRM(unit.spikes, unit.position)            
            if np.nanpercentile(rm,Def.wholetrack_imshow_pct_cutoff) >= 1.:
                print(clustname)
                population[unit.name] = unit

C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:178: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:178: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:184: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:184: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\utility_fx.py:326: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\utility_fx.py:499: RuntimeWarning: divide by zero encountered in rec

TT1\cl-maze1.1
TT1\cl-maze1.2
TT1\cl-maze1.3
TT12\cl-maze1.1
TT12\cl-maze1.3
TT15\cl-maze1.1
TT15\cl-maze1.2
TT5\cl-maze1.5
TT6\cl-maze1.1
TT6\cl-maze1.2
TT6\cl-maze1.3


# kNearest Neighbors Classification

## Step-by-step: 1 CV iter, Single Alley k-Nearest Neighbors Decoding of Texture.
#### Multiple kNN classifiers created, one for each sliding window along the trials

### Create population feature matrix + label vector

In [107]:
alley = 1
repFx = createVisitSummaryFeatures
X, Y = setupAlleyData(alley, repFx)

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\ndimage\measurements.py:1359: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


### Separate Trials Into Train and Test

In [ ]:
# this is all done once for a whole cv routine. these are static params.
numNeighbors = 5
desiredWindow = numNeighbors * 3 # meaning without the downsampling occuring because of the train/test split
ptrain = 0.8
wnSize = int(round(desiredWindow + (desiredWindow*(1.-ptrain)))) # say 15 is hardcoded so there are on average 5 trials each of A,B,C. Increase size to account for train/test downsampling 
wnStep = 2

nTrials = len(unit.alleys[1]) # all alleys should have same number of laps
ntrain, ntest = round(nTrials*ptrain), round(nTrials*(1.-ptrain))

wins = []
l = list(range(nTrials))
for i in range(int(np.ceil(len(l)/wnStep))):
    wins.append(l[0+(wnStep*i):wnSize+(wnStep*i)])
wins = [i for i in wins if len(i)>=wnSize] 
bounds = [(i[0],i[-1]) for i in wins]

# this is done for each cv iteration as it's here we are actually splitting trials into train and test
trTrials = np.sort(np.random.choice(range(nTrials), ntrain, replace=False))
teTrials = np.sort(np.asarray([i for i in range(nTrials) if i not in trTrials]))

### Loop over each window of trials. Train a classifer. Test on test trials, store results (prediction, [kNeighbors distances])

In [240]:
knn_classifiers = []
test_output = {te:{i:{'kn':None,'prediction':None, 'actual':Y[te]} for i in range(len(bounds))} for te in teTrials}

for i,interval in enumerate(bounds):
    sample = trTrials[np.logical_and(trTrials>interval[0], trTrials<=interval[1])]
    Xsample, Ysample = X[sample,:], Y[sample,:]
    clf = neighbors.KNeighborsClassifier(numNeighbors)
    clf.fit(Xsample, Ysample.ravel()) # it complains if Y isn't (n,) 
    knn_classifiers.append(clf)
    for te in teTrials:
        test_output[te][i]['prediction'] = clf.predict([X[te]])
        test_output[te][i]['kn'] = clf.kneighbors([X[te]])[0]

### Find for each test trial the window in which the kNNs are closest and see what prediction that classifier gives. 
### Compare to real label. Get % correct

In [253]:
perf = []

for te in teTrials:
    avgkNNdist = [np.mean(test_output[te][i]['kn']) for i in range(len(bounds))]
    preds = [test_output[te][i]['prediction'] for i in range(len(bounds))]
    real = [test_output[te][i]['actual'] for i in range(len(bounds))]
    idx = np.argmin(avgkNNdist)
    outcome = preds[idx] == real[idx]
    perf.append(outcome)

## Routine: Single Alley CV Performance of windowed kNN Classification

In [438]:
# Parameter set up for whole CV run
alley = 16
ncv = 1000
repFx = createVisitSummaryFeatures
X, Y = setupAlleyData(alley, repFx)
perfs = []

numNeighbors = 3
desiredWindow = numNeighbors * 3 # meaning without the downsampling occuring because of the train/test split
ptrain = 0.8
wnSize = int(round(desiredWindow + (desiredWindow*(1.-ptrain)))) # say 15 is hardcoded so there are on average 5 trials each of A,B,C. Increase size to account for train/test downsampling 
wnStep = 2

nTrials = len(unit.alleys[alley]) # all alleys should have same number of laps
ntrain, ntest = round(nTrials*ptrain), round(nTrials*(1.-ptrain))

wins = []
l = list(range(nTrials))
for i in range(int(np.ceil(len(l)/wnStep))):
    wins.append(l[0+(wnStep*i):wnSize+(wnStep*i)])
wins = [i for i in wins if len(i)>=wnSize] 
bounds = [(i[0],i[-1]) for i in wins]

# loop over each cv iter, run windowed kNN, get overall perf for that run.
for n in range(ncv):
    trTrials = np.sort(np.random.choice(range(nTrials), ntrain, replace=False))
    teTrials = np.sort(np.asarray([i for i in range(nTrials) if i not in trTrials]))
    
    knn_classifiers = []
    test_output = {te:{i:{'kn':None,'prediction':None, 'actual':Y[te]} for i in range(len(bounds))} for te in teTrials}

    for i,interval in enumerate(bounds):
        sample = trTrials[np.logical_and(trTrials>interval[0], trTrials<=interval[1])]
        Xsample, Ysample = X[sample,:], Y[sample,:]
        clf = neighbors.KNeighborsClassifier(numNeighbors)
        clf.fit(Xsample, Ysample.ravel()) # it complains if Y isn't (n,) 
        knn_classifiers.append(clf)
        for te in teTrials:
            test_output[te][i]['prediction'] = clf.predict([X[te]])
            test_output[te][i]['kn'] = clf.kneighbors([X[te]])[0]
    perf = []

    for te in teTrials:
        avgkNNdist = [np.nanmean(test_output[te][i]['kn']) for i in range(len(bounds))]
        preds = [test_output[te][i]['prediction'] for i in range(len(bounds))]
        real = [test_output[te][i]['actual'] for i in range(len(bounds))]
        idx = np.argmin(avgkNNdist)
        outcome = preds[idx] == real[idx]
        perf.append(outcome)
    
    perfs.append(sum(perf)/len(perf))
        
perfs = np.asarray(pefs)

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\ndimage\measurements.py:1359: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]


NameError: name 'pefs' is not defined

### Function: CV run for a single alley, passed as args. All params hardcoded inside, change by hand in code. Returns perfs as (ncv,1) array

In [57]:
def singleAlley_kNNRun(alley):
    # Parameter set up for whole CV run
    ncv = 500
    repFx = createVisitSummaryFeatures
    X, Y = setupAlleyData(alley, repFx)
    perfs = []

    numNeighbors = 3
    desiredWindow = numNeighbors * 3 # meaning without the downsampling occuring because of the train/test split
    ptrain = 0.7
    wnSize = int(round(desiredWindow + (desiredWindow*(1.-ptrain)))) # say 15 is hardcoded so there are on average 5 trials each of A,B,C. Increase size to account for train/test downsampling 
    wnStep = 1

    nTrials = len(unit.alleys[alley])
    ntrain, ntest = round(nTrials*ptrain), round(nTrials*(1.-ptrain))

    wins = []
    l = list(range(nTrials))
    for i in range(int(np.ceil(len(l)/wnStep))):
        wins.append(l[0+(wnStep*i):wnSize+(wnStep*i)])
    wins = [i for i in wins if len(i)>=wnSize] 
    bounds = [(i[0],i[-1]) for i in wins]

    # loop over each cv iter, run windowed kNN, get overall perf for that run.
    for n in range(ncv):
        trTrials = np.sort(np.random.choice(range(nTrials), ntrain, replace=False))
        teTrials = np.sort(np.asarray([i for i in range(nTrials) if i not in trTrials]))

        knn_classifiers = []
        test_output = {te:{i:{'kn':1e6,'prediction':None, 'actual':Y[te]} for i in range(len(bounds))} for te in teTrials}

        for i,interval in enumerate(bounds):
            sample = trTrials[np.logical_and(trTrials>interval[0], trTrials<=interval[1])]
            Xsample, Ysample = X[sample,:], Y[sample]
            clf = neighbors.KNeighborsClassifier(numNeighbors)
            clf.fit(Xsample, Ysample.ravel()) # it complains if Y isn't (n,) 
            knn_classifiers.append(clf)
            for te in teTrials:
                try:
                    test_output[te][i]['prediction'] = clf.predict([X[te]])
                    test_output[te][i]['kn'] = clf.kneighbors([X[te]])[0]
                except:
                    pass

        perf = []

        for te in teTrials:
            avgkNNdist = [np.mean(test_output[te][i]['kn']) for i in range(len(bounds))]
            preds = [test_output[te][i]['prediction'] for i in range(len(bounds))]
            real = [test_output[te][i]['actual'] for i in range(len(bounds))]
            idx = np.argmin(avgkNNdist)
            if preds[idx] is not None:
                outcome = preds[idx] == real[idx]
                perf.append(outcome)

        perfs.append(sum(perf)/len(perf))

    perfs = np.asarray(perfs)

    return perfs

In [58]:
alley_perfs = {i:None for i in beltwayAlleys}

for alley in beltwayAlleys:
    print(alley)
    perfs = singleAlley_kNNRun(alley)
    alley_perfs[alley] = perfs

16


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\ndimage\measurements.py:1359: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]


17


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


3
1
5
7
8
10
11


# Random Forest Classification - whole Pop vector, 1 alley at a time

In [85]:
def runRandomForest(X, Y, runs=300, trees=700):
    oobs = []
    fimps = []
    paths = []
    for i in range(runs):
        print(i)
        clf = RandomForestClassifier(n_estimators=trees, 
                                     oob_score=True,
                                     max_features = None,
                                     max_depth = 4
                                    )       
        clf.fit(X,Y)
        oobs.append(clf.oob_score_)
        fimps.append(clf.feature_importances_)
        paths.append(clf.decision_path(X))
        
    return oobs, fimps, paths

In [706]:
# def runRandomForest(X, Y, runs=300, trees=700):
#     oobs, precisions, recalls, f1s, accuracies = [], [], [], [],[]
#     avgType = 'macro'
#     for i in range(runs):
#         print(i)
#         clf = RandomForestClassifier(n_estimators=trees, 
#                                      oob_score=True,
#                                      max_features = None,
#                                      max_depth = 4
#                                     )
        
#         clf2 = RandomForestClassifier(n_estimators=trees, 
#                                      oob_score=True,
#                                      max_features = None,
#                                      max_depth = 4
#                                     )
#         Xtrain, Xtest, ytrain, ytest = train_test_split(X,Y,shuffle=True)
#         clf.fit(Xtrain,ytrain)
#         clf2.fit(X,Y)
#         oobs.append(clf2.oob_score_)
#         yfit = clf.predict(Xtest)
#         p = precision_score(ytest, yfit, average=avgType)
#         r = recall_score(ytest, yfit, average=avgType)
#         f1 = f1_score(ytest, yfit, average=avgType)
#         acc = accuracy_score(ytest,yfit)
#         precisions.append(p)
#         recalls.append(r)
#         f1s.append(f1)
#         accuracies.append(acc)
#     return [oobs, precisions, recalls, f1s, accuracies]

In [12]:
def randomForest_wrapper(alley,repFx):
    """
    Wrapper function that takes a target alley
    to decode texture present. Creates data and label matrices X,Y
    and runs a random forest classifier with a set
    number of trees and runs. Returns readouts:
    oob score, precision, recall, F1, accuracy for each run
    Plots the histograms of each on a single new figure
    """
    X,Y = setupAlleyData(alley,repFx)
    oobs, precisions, recalls, f1s, accuracies = runRandomForest(X,Y, runs=100)
    fig, ax = plt.subplots()
    ax.hist(precisions, color='b', alpha=0.5)
    ax.hist(recalls, color='r', alpha=0.5)
    ax.hist(f1s, color='g', alpha=0.5)
    ax.hist(accuracies, color='k', alpha=0.5)
    ax.hist(oobs,color='purple')
    ax.legend(["Precision", "Recall", "F1 Score", "Accuracy", "OOB"])
    ax.set_ylabel("Frquency")
    ax.set_xlabel("Performance")
    ax.set_title(alley)
    plt.show()
    return oobs, precisions, recalls, f1s, accuracies

In [32]:
alloobs, allfimps = [], []

parmdict = {
    'nRuns':5000, 
    'nTrees':1000, 
    'Max Depth':None, 
    'Max Feats':'auto',
    'Cell inclusion in population': '95pctile >=1Hz overall',
    'Visit inclusion in data matrix': 'Mean alley activity >=1 Hz'
    'Bootstrap: Trials'
    }

features = [
    'time',
    'auc',
    'com',
    'comval',
    'max95',
    'locmax95',
    'mean',
    'avgdrds',
    'maxdrds'
           ]

datadict = {i:{'X':None, 'Y':None} for i in beltwayAlleys}
for alley in beltwayAlleys:
    print(alley)
    alleyoobs, alleyfimps = [], []
    datadict[alley]['X'], datadict[alley]['Y'] = setupAlleyData(alley, createVisitSummaryFeatures, features, bootstrap=False)
    for run in range(parmdict['nRuns']):
        print(alley,run)
        idx = np.random.choice(range(datadict[alley]['X'].shape[0]), datadict[alley]['X'].shape[0], replace=True)
        Xsample, Ysample = datadict[alley]['X'][idx,:], datadict[alley]['Y'][idx]

        # adjust clf params that need to know alley specific info, e.g. X shape
        if parmdict['Max Depth'] is None:
            maxd = None
        elif type(parmdict['Max Depth'])==int:
            maxd = parmdict['Max Depth']
        else:
            maxd = round(len(features)*parmdict['Max Depth'])

        clf = RandomForestClassifier(n_estimators=parmdict['nTrees'], 
                                     oob_score=True,
                                     max_features = parmdict['Max Feats'],
                                     max_depth = maxd
                                    ) 
        clf.fit(Xsample, Ysample)
        #alleyfimps.append(clf.feature_importances_)
        alleyoobs.append(clf.oob_score_)
    alloobs.append(alleyoobs)
    #allfimps.append(alleyfimps)
        

16
16 0
16 1
16 2
16 3
16 4
16 5
16 6
16 7
16 8
16 9
16 10
16 11
16 12
16 13
16 14
16 15
16 16
16 17
16 18
16 19
16 20
16 21
16 22
16 23
16 24
16 25
16 26
16 27
16 28
16 29
16 30
16 31
16 32
16 33
16 34
16 35
16 36
16 37
16 38
16 39
16 40
16 41
16 42
16 43
16 44
16 45
16 46
16 47
16 48
16 49
16 50
16 51
16 52
16 53
16 54
16 55
16 56
16 57
16 58
16 59
16 60
16 61
16 62
16 63
16 64
16 65
16 66
16 67
16 68
16 69
16 70
16 71
16 72
16 73
16 74
16 75
16 76
16 77
16 78
16 79
16 80
16 81
16 82
16 83
16 84
16 85
16 86
16 87
16 88
16 89
16 90
16 91
16 92
16 93
16 94
16 95
16 96
16 97
16 98
16 99
16 100
16 101
16 102
16 103
16 104
16 105
16 106
16 107
16 108
16 109
16 110
16 111
16 112
16 113
16 114
16 115
16 116
16 117
16 118
16 119
16 120
16 121
16 122
16 123
16 124
16 125
16 126
16 127
16 128
16 129
16 130
16 131
16 132
16 133
16 134
16 135
16 136
16 137
16 138
16 139
16 140
16 141
16 142
16 143
16 144
16 145
16 146
16 147
16 148
16 149
16 150
16 151
16 152
16 153
16 154
16 155
16 156
16 157
1

16 1163
16 1164
16 1165
16 1166
16 1167
16 1168
16 1169
16 1170
16 1171
16 1172
16 1173
16 1174
16 1175
16 1176
16 1177
16 1178
16 1179
16 1180
16 1181
16 1182
16 1183
16 1184
16 1185
16 1186
16 1187
16 1188
16 1189
16 1190
16 1191
16 1192
16 1193
16 1194
16 1195
16 1196
16 1197
16 1198
16 1199
16 1200
16 1201
16 1202
16 1203
16 1204
16 1205
16 1206
16 1207
16 1208
16 1209
16 1210
16 1211
16 1212
16 1213
16 1214
16 1215
16 1216
16 1217
16 1218
16 1219
16 1220
16 1221
16 1222
16 1223
16 1224
16 1225
16 1226
16 1227
16 1228
16 1229
16 1230
16 1231
16 1232
16 1233
16 1234
16 1235
16 1236
16 1237
16 1238
16 1239
16 1240
16 1241
16 1242
16 1243
16 1244
16 1245
16 1246
16 1247
16 1248
16 1249
16 1250
16 1251
16 1252
16 1253
16 1254
16 1255
16 1256
16 1257
16 1258
16 1259
16 1260
16 1261
16 1262
16 1263
16 1264
16 1265
16 1266
16 1267
16 1268
16 1269
16 1270
16 1271
16 1272
16 1273
16 1274
16 1275
16 1276
16 1277
16 1278
16 1279
16 1280
16 1281
16 1282
16 1283
16 1284
16 1285
16 1286
16 1287


16 2188
16 2189
16 2190
16 2191
16 2192
16 2193
16 2194
16 2195
16 2196
16 2197
16 2198
16 2199
16 2200
16 2201
16 2202
16 2203
16 2204
16 2205
16 2206
16 2207
16 2208
16 2209
16 2210
16 2211
16 2212
16 2213
16 2214
16 2215
16 2216
16 2217
16 2218
16 2219
16 2220
16 2221
16 2222
16 2223
16 2224
16 2225
16 2226
16 2227
16 2228
16 2229
16 2230
16 2231
16 2232
16 2233
16 2234
16 2235
16 2236
16 2237
16 2238
16 2239
16 2240
16 2241
16 2242
16 2243
16 2244
16 2245
16 2246
16 2247
16 2248
16 2249
16 2250
16 2251
16 2252
16 2253
16 2254
16 2255
16 2256
16 2257
16 2258
16 2259
16 2260
16 2261
16 2262
16 2263
16 2264
16 2265
16 2266
16 2267
16 2268
16 2269
16 2270
16 2271
16 2272
16 2273
16 2274
16 2275
16 2276
16 2277
16 2278
16 2279
16 2280
16 2281
16 2282
16 2283
16 2284
16 2285
16 2286
16 2287
16 2288
16 2289
16 2290
16 2291
16 2292
16 2293
16 2294
16 2295
16 2296
16 2297
16 2298
16 2299
16 2300
16 2301
16 2302
16 2303
16 2304
16 2305
16 2306
16 2307
16 2308
16 2309
16 2310
16 2311
16 2312


16 3213
16 3214
16 3215
16 3216
16 3217
16 3218
16 3219
16 3220
16 3221
16 3222
16 3223
16 3224
16 3225
16 3226
16 3227
16 3228
16 3229
16 3230
16 3231
16 3232
16 3233
16 3234
16 3235
16 3236
16 3237
16 3238
16 3239
16 3240
16 3241
16 3242
16 3243
16 3244
16 3245
16 3246
16 3247
16 3248
16 3249
16 3250
16 3251
16 3252
16 3253
16 3254
16 3255
16 3256
16 3257
16 3258
16 3259
16 3260
16 3261
16 3262
16 3263
16 3264
16 3265
16 3266
16 3267
16 3268
16 3269
16 3270
16 3271
16 3272
16 3273
16 3274
16 3275
16 3276
16 3277
16 3278
16 3279
16 3280
16 3281
16 3282
16 3283
16 3284
16 3285
16 3286
16 3287
16 3288
16 3289
16 3290
16 3291
16 3292
16 3293
16 3294
16 3295
16 3296
16 3297
16 3298
16 3299
16 3300
16 3301
16 3302
16 3303
16 3304
16 3305
16 3306
16 3307
16 3308
16 3309
16 3310
16 3311
16 3312
16 3313
16 3314
16 3315
16 3316
16 3317
16 3318
16 3319
16 3320
16 3321
16 3322
16 3323
16 3324
16 3325
16 3326
16 3327
16 3328
16 3329
16 3330
16 3331
16 3332
16 3333
16 3334
16 3335
16 3336
16 3337


16 4238
16 4239
16 4240
16 4241
16 4242
16 4243
16 4244
16 4245
16 4246
16 4247
16 4248
16 4249
16 4250
16 4251
16 4252
16 4253
16 4254
16 4255
16 4256
16 4257
16 4258
16 4259
16 4260
16 4261
16 4262
16 4263
16 4264
16 4265
16 4266
16 4267
16 4268
16 4269
16 4270
16 4271
16 4272
16 4273
16 4274
16 4275
16 4276
16 4277
16 4278
16 4279
16 4280
16 4281
16 4282
16 4283
16 4284
16 4285
16 4286
16 4287
16 4288
16 4289
16 4290
16 4291
16 4292
16 4293
16 4294
16 4295
16 4296
16 4297
16 4298
16 4299
16 4300
16 4301
16 4302
16 4303
16 4304
16 4305
16 4306
16 4307
16 4308
16 4309
16 4310
16 4311
16 4312
16 4313
16 4314
16 4315
16 4316
16 4317
16 4318
16 4319
16 4320
16 4321
16 4322
16 4323
16 4324
16 4325
16 4326
16 4327
16 4328
16 4329
16 4330
16 4331
16 4332
16 4333
16 4334
16 4335
16 4336
16 4337
16 4338
16 4339
16 4340
16 4341
16 4342
16 4343
16 4344
16 4345
16 4346
16 4347
16 4348
16 4349
16 4350
16 4351
16 4352
16 4353
16 4354
16 4355
16 4356
16 4357
16 4358
16 4359
16 4360
16 4361
16 4362


...

11 316
11 317
11 318
11 319
11 320
11 321
11 322
11 323
11 324
11 325
11 326
11 327
11 328
11 329
11 330
11 331
11 332
11 333
11 334
11 335
11 336
11 337
11 338
11 339
11 340
11 341
11 342
11 343
11 344
11 345
11 346
11 347
11 348
11 349
11 350
11 351
11 352
11 353
11 354
11 355
11 356
11 357
11 358
11 359
11 360
11 361
11 362
11 363
11 364
11 365
11 366
11 367
11 368
11 369
11 370
11 371
11 372
11 373
11 374
11 375
11 376
11 377
11 378
11 379
11 380
11 381
11 382
11 383
11 384
11 385
11 386
11 387
11 388
11 389
11 390
11 391
11 392
11 393
11 394
11 395
11 396
11 397
11 398
11 399
11 400
11 401
11 402
11 403
11 404
11 405
11 406
11 407
11 408
11 409
11 410
11 411
11 412
11 413
11 414
11 415
11 416
11 417
11 418
11 419
11 420
11 421
11 422
11 423
11 424
11 425
11 426
11 427
11 428
11 429
11 430
11 431
11 432
11 433
11 434
11 435
11 436
11 437
11 438
11 439
11 440
11 441
11 442
11 443
11 444
11 445
11 446
11 447
11 448
11 449
11 450
11 451
11 452
11 453
11 454
11 455
11 456
11 457
11 458

11 1426
11 1427
11 1428
11 1429
11 1430
11 1431
11 1432
11 1433
11 1434
11 1435
11 1436
11 1437
11 1438
11 1439
11 1440
11 1441
11 1442
11 1443
11 1444
11 1445
11 1446
11 1447
11 1448
11 1449
11 1450
11 1451
11 1452
11 1453
11 1454
11 1455
11 1456
11 1457
11 1458
11 1459
11 1460
11 1461
11 1462
11 1463
11 1464
11 1465
11 1466
11 1467
11 1468
11 1469
11 1470
11 1471
11 1472
11 1473
11 1474
11 1475
11 1476
11 1477
11 1478
11 1479
11 1480
11 1481
11 1482
11 1483
11 1484
11 1485
11 1486
11 1487
11 1488
11 1489
11 1490
11 1491
11 1492
11 1493
11 1494
11 1495
11 1496
11 1497
11 1498
11 1499
11 1500
11 1501
11 1502
11 1503
11 1504
11 1505
11 1506
11 1507
11 1508
11 1509
11 1510
11 1511
11 1512
11 1513
11 1514
11 1515
11 1516
11 1517
11 1518
11 1519
11 1520
11 1521
11 1522
11 1523
11 1524
11 1525
11 1526
11 1527
11 1528
11 1529
11 1530
11 1531
11 1532
11 1533
11 1534
11 1535
11 1536
11 1537
11 1538
11 1539
11 1540
11 1541
11 1542
11 1543
11 1544
11 1545
11 1546
11 1547
11 1548
11 1549
11 1550


11 2451
11 2452
11 2453
11 2454
11 2455
11 2456
11 2457
11 2458
11 2459
11 2460
11 2461
11 2462
11 2463
11 2464
11 2465
11 2466
11 2467
11 2468
11 2469
11 2470
11 2471
11 2472
11 2473
11 2474
11 2475
11 2476
11 2477
11 2478
11 2479
11 2480
11 2481
11 2482
11 2483
11 2484
11 2485
11 2486
11 2487
11 2488
11 2489
11 2490
11 2491
11 2492
11 2493
11 2494
11 2495
11 2496
11 2497
11 2498
11 2499
11 2500
11 2501
11 2502
11 2503
11 2504
11 2505
11 2506
11 2507
11 2508
11 2509
11 2510
11 2511
11 2512
11 2513
11 2514
11 2515
11 2516
11 2517
11 2518
11 2519
11 2520
11 2521
11 2522
11 2523
11 2524
11 2525
11 2526
11 2527
11 2528
11 2529
11 2530
11 2531
11 2532
11 2533
11 2534
11 2535
11 2536
11 2537
11 2538
11 2539
11 2540
11 2541
11 2542
11 2543
11 2544
11 2545
11 2546
11 2547
11 2548
11 2549
11 2550
11 2551
11 2552
11 2553
11 2554
11 2555
11 2556
11 2557
11 2558
11 2559
11 2560
11 2561
11 2562
11 2563
11 2564
11 2565
11 2566
11 2567
11 2568
11 2569
11 2570
11 2571
11 2572
11 2573
11 2574
11 2575


11 3476
11 3477
11 3478
11 3479
11 3480
11 3481
11 3482
11 3483
11 3484
11 3485
11 3486
11 3487
11 3488
11 3489
11 3490
11 3491
11 3492
11 3493
11 3494
11 3495
11 3496
11 3497
11 3498
11 3499
11 3500
11 3501
11 3502
11 3503
11 3504
11 3505
11 3506
11 3507
11 3508
11 3509
11 3510
11 3511
11 3512
11 3513
11 3514
11 3515
11 3516
11 3517
11 3518
11 3519
11 3520
11 3521
11 3522
11 3523
11 3524
11 3525
11 3526
11 3527
11 3528
11 3529
11 3530
11 3531
11 3532
11 3533
11 3534
11 3535
11 3536
11 3537
11 3538
11 3539
11 3540
11 3541
11 3542
11 3543
11 3544
11 3545
11 3546
11 3547
11 3548
11 3549
11 3550
11 3551
11 3552
11 3553
11 3554
11 3555
11 3556
11 3557
11 3558
11 3559
11 3560
11 3561
11 3562
11 3563
11 3564
11 3565
11 3566
11 3567
11 3568
11 3569
11 3570
11 3571
11 3572
11 3573
11 3574
11 3575
11 3576
11 3577
11 3578
11 3579
11 3580
11 3581
11 3582
11 3583
11 3584
11 3585
11 3586
11 3587
11 3588
11 3589
11 3590
11 3591
11 3592
11 3593
11 3594
11 3595
11 3596
11 3597
11 3598
11 3599
11 3600


11 4501
11 4502
11 4503
11 4504
11 4505
11 4506
11 4507
11 4508
11 4509
11 4510
11 4511
11 4512
11 4513
11 4514
11 4515
11 4516
11 4517
11 4518
11 4519
11 4520
11 4521
11 4522
11 4523
11 4524
11 4525
11 4526
11 4527
11 4528
11 4529
11 4530
11 4531
11 4532
11 4533
11 4534
11 4535
11 4536
11 4537
11 4538
11 4539
11 4540
11 4541
11 4542
11 4543
11 4544
11 4545
11 4546
11 4547
11 4548
11 4549
11 4550
11 4551
11 4552
11 4553
11 4554
11 4555
11 4556
11 4557
11 4558
11 4559
11 4560
11 4561
11 4562
11 4563
11 4564
11 4565
11 4566
11 4567
11 4568
11 4569
11 4570
11 4571
11 4572
11 4573
11 4574
11 4575
11 4576
11 4577
11 4578
11 4579
11 4580
11 4581
11 4582
11 4583
11 4584
11 4585
11 4586
11 4587
11 4588
11 4589
11 4590
11 4591
11 4592
11 4593
11 4594
11 4595
11 4596
11 4597
11 4598
11 4599
11 4600
11 4601
11 4602
11 4603
11 4604
11 4605
11 4606
11 4607
11 4608
11 4609
11 4610
11 4611
11 4612
11 4613
11 4614
11 4615
11 4616
11 4617
11 4618
11 4619
11 4620
11 4621
11 4622
11 4623
11 4624
11 4625


In [37]:
plt.figure()
plt.bar(range(len(alloobs)), [np.mean(i) for i in alloobs], yerr=[3*np.std(i) for i in alloobs],width=0.3)
plt.hlines(1/3,0,8)
for i,a in enumerate(alloobs):
    plt.scatter([i]*len(a), a, alpha=0.6, zorder=5)

In [85]:
fig, ax = plt.subplots(3,3, sharey=True)
for i,alley in enumerate(beltwayAlleys):
    X,Y = datadict[alley]['X'], datadict[alley]['Y']
    fn = list(itertools.repeat(features,int(X.shape[1]/len(features))))
    fn = [item for sublist in fn for item in sublist]
    print(X.shape[1],len(fn))
    fi = np.asarray(allfimps[i])
    fig.axes[i].bar(range(fi.shape[1]), np.mean(fi,axis=0), yerr=np.std(fi,axis=0))
    fig.axes[i].vlines(np.arange(0,X.shape[1]+len(features),len(features))-0.3,0,.2)
    fig.axes[i].set_xticks(range(len(fn)))
    fig.axes[i].set_xticklabels(fn,rotation=90,fontsize=10)
    fig.axes[i].set_ylabel(f"Alley {i+1}")
plt.suptitle("Feature Importances",fontsize=20)
plt.tight_layout()

7 0


IndexError: tuple index out of range

In [ ]:
# R859 BRD5 units passing permutation test 5/4/20
goodunits = ['TT1cl-maze1.1','TT1cl-maze1.3','TT1cl-maze1.4','TT1cl-maze1.6','TT6cl-maze1.7','TT13cl-maze1.3']

In [24]:
def genParmStr(parmdict):
    parmstring = (",").join([f"{k}:{v}" for k,v in parmdict.items()])
    return parmstring

In [ ]:
class ParameterManager():
    
    def __init__(parmdict, features):
        self.parmdict = parmdict
        self.features = features
        self.code = self.genCode()
        
    def save(self, df='E:\\Ratterdam\\decoder_params\\'):
        with open(df+self.code,'r') as file:
            for k,v in self.parmdict.items():
                file.write(f"{k}:{v}\n")
            for feat in self.features:
                file.write(f"{feat}\n")
    
    def genCode(self):
        d = datetime.datetime.now()
        self.code = f"{d.year}-{d.month}-{d.day}--{d.hour}-{d.minute}-{d.second}"
                
        

In [522]:
# Jackknife (kfold split=n) 

features = [
    'boundMaxs', #nb this is three separate features but they come as a set conceptually
    'auc',
    'com',
    'comval',
    'max95',
    'locmax95',
    'mean',
    'avgdrds',
    'maxdrds'
           ]

parmdict = {
    'nRuns':'n', 
    'nTrees':25000, 
    'Max Depth':15, 
    'Max Feats':'auto',
    'Run type':'Kfold',
    'Cell inclusion': '95pctile >=1Hz overall',
    'Alley inclusion': 'Mean alley activity >=1 Hz',
    'Bootstrap':None,
    'Test size':'Na'
    }


scores=[]
oobs = []
for alley in beltwayAlleys:
    print(alley)
    ascores = []
    aoobs = []
    X,Y = setupAlleyData(alley, createVisitSummaryFeatures, features, bootstrap=False)
    kf = KFold(n_splits=X.shape[0], shuffle=True)
    c=0
    for train, test in kf.split(X):
        print(c+1)
        c +=1
        Xtrain,Xtest =X[train], X[test]
        Ytrain, Ytest = Y[train],Y[test]
        clf = RandomForestClassifier(n_estimators=parmdict['nTrees'], 
                                         oob_score=True,
                                         max_features = parmdict['Max Feats'],
                                         max_depth = parmdict['Max Depth']
                                        ) 
        clf.fit(Xtrain,Ytrain)
        ascores.append(clf.score(Xtest, Ytest))
        aoobs.append(clf.oob_score_)
    scores.append(ascores)
    oobs.append(aoobs)
        

16
1


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\ndimage\measurements.py:1359: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  >>> np.delete(arr, [1,3,5], None)


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
17
1


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  >>> np.delete(arr, [1,3,5], None)


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
3
1


...

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\ndimage\measurements.py:1359: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  >>> np.delete(arr, [1,3,5], None)


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
5
1


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  >>> np.delete(arr, [1,3,5], None)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\ndimage\measurements.py:1359: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
7


ValueError: Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required by StandardScaler.

In [526]:
data = oobs
plt.figure()
plt.bar(range(len(data)), [np.mean(i) for i in data], yerr=[2.54*np.std(i) for i in data],width=0.3)
plt.hlines(1/3,0,8)
for i,a in enumerate(data):
    plt.scatter([i]*len(a), a, alpha=0.6, zorder=5)
string = parmString()
plt.text(0.01, 0.1, string, fontsize=6, transform=plt.gcf().transFigure)

Text(0.01,0.1,'nRuns:n\nnTrees:25000\nMax Depth:15\nMax Feats:auto\nRun type:Kfold\nCell inclusion:95pctile >=1Hz overall\nAlley inclusion:Mean alley activity >=1 Hz\nBootstrap:None\nTest size:Na\nboundMaxs\nauc\ncom\ncomval\nmax95\nlocmax95\nmean\navgdrds\nmaxdrds\n')

In [27]:
# V_IJ Unbiased (Wagner 2014, forestci implementation)

features = [
    'time',
    'auc',
    'com',
    'comval',
    'max95',
    'locmax95',
    'mean',
    'avgdrds',
    'maxdrds'
           ]

parmdict = {
    'nRuns':6, 
    'nTrees':5000, 
    'Max Depth':6, 
    'Max Feats':'auto',
    'Run type':'Kfold',
    'Cell inclusion in population': '95pctile >=1Hz overall',
    'Visit inclusion in data matrix': 'Mean alley activity >=1 Hz'
    'Bootstrap: Cells in original pop dict, subject to alley activity check'
    }

# split the datainto training and test set
alley = 16
X,Y = setupAlleyData(alley, createVisitSummaryFeatures, features, bootstrap=False)
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,shuffle=True)

# create RandomForestClassifier
RFC = RandomForestClassifier(max_features=parmdict['Max Feats'], 
                             n_estimators=parmdict['nTrees'],
                             oob_score = True,
                             random_state=42)

RFC.fit(X_train, y_train)
y_hat = RFC.predict_proba(X_test)

# Calculate the variance
spam_V_IJ_unbiased = fci.random_forest_error(RFC, X_train, X_test)

No calibration with n_samples <= 20


In [135]:
def convertStrIntLabel(label):
    conv = {'A':0.,'B':1.,'C':2.}
    return conv[label]

In [290]:
# All data, then get array of individual tree oob scores and gen ci/errorbars 

#nb: after doing this it doesnt work conceptually. You can get the oob sample for each row of X meaning the trees for which that row was held out and get a majority vote of class
# but youre just left with n (num trials) outcomes correct/incorrect vote. 1) thats categorical not numeric and 2) it's not the std of the overall oob score its just the data that
# generates it. you need multiple final oob scores themsevles so that the std is of the oob. So youre left with 1) bootstrap + get accuracy 2) MC CV and do oob+axx. 

features = [
    'boundMaxs', #nb this is three separate features but they come as a set conceptually
    'auc',
    'com',
    'comval',
    'max95',
    'locmax95',
    'mean',
    'avgdrds',
    'maxdrds'
           ]

parmdict = {
    'nRuns':1, 
    'nTrees':20000, 
    'Max Depth':15, 
    'Max Feats':'auto',
    'Run type':'All data once',
    'Cell inclusion in population': '95pctile >=1Hz overall',
    'Visit inclusion in data matrix': 'Mean alley activity >=1 Hz'
    'Bootstrap: None'
    }
oobs = []
oobvars = []
for alley in beltwayAlleys:
    suboobs = []
    print(alley)
    X,Y = setupAlleyData(alley, createVisitSummaryFeatures, features, bootstrap=False)
    clf = RandomForestClassifier(n_estimators=parmdict['nTrees'], 
                                     oob_score=True,
                                     max_features = parmdict['Max Feats'],
                                     max_depth = parmdict['Max Depth']
                                    ) 
    clf.fit(X,Y)
    oobs.append(clf.oob_score_)
    
#     n_samples = X.shape[0]
#     for i,(sample,label) in enumerate(zip(X,Y)):
#         label = convertStrIntLabels(label)
#         sample = sample.reshape(1,-1)
#         predictions = []
#         for tree in clf.estimators_:
#             unsampled_indices = _generate_unsampled_indices(tree.random_state,n_samples,n_samples)
#             if i in unsampled_indices:
#                 outcome = tree.predict(sample)
#                 predictions.append(outcome)
#         vote = scipy.stats.mode(predictions)
#         suboobs.append(sum(label==vote[0][0][0])/
#     oobvars.append(suboobs)
        

16
17


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  >>> np.delete(arr, [1,3,5], None)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\ndimage\measurements.py:1359: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]


3


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\ndimage\measurements.py:1359: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  >>> np.delete(arr, [1,3,5], None)


1


...

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\ndimage\measurements.py:1359: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  >>> np.delete(arr, [1,3,5], None)


10


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  >>> np.delete(arr, [1,3,5], None)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\ndimage\measurements.py:1359: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]


11


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  >>> np.delete(arr, [1,3,5], None)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\ndimage\measurements.py:1359: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]


In [292]:
plt.figure()
plt.bar(range(len(oobs)), oobs,width=0.3)
plt.hlines(1/3,0,8)

In [224]:
# Monte Carlo CV and get accuracy / oob + error estimates

features = [
    'boundMaxs', #nb this is three separate features but they come as a set conceptually
    'auc',
    'com',
    'comval',
    'max95',
    'locmax95',
    'mean',
    'avgdrds',
    'maxdrds'
           ]

parmdict = {
    'nRuns':25, 
    'nTrees':15000, 
    'Max Depth':15, 
    'Max Feats':'auto',
    'Run type':'Monte Carlo CV',
    'Cell inclusion in population': '95pctile >=1Hz overall',
    'Visit inclusion in data matrix': 'Mean alley activity >=1 Hz'
    'Bootstrap: None',
    'Test size':0.2
    }


scores=[]
oobs = []
for alley in beltwayAlleys:
    print(alley)
    ascores = []
    aoobs = []
    X,Y = setupAlleyData(alley, createVisitSummaryFeatures, features, bootstrap=False)
    c=0
    for i in range(parmdict['nRuns']):
        print(c+1)
        c +=1
        Xtrain, Xtest, ytrain, ytest = train_test_split(X,Y,test_size=parmdict['Test size'],shuffle=True)
        clf = RandomForestClassifier(n_estimators=parmdict['nTrees'], 
                                         oob_score=True,
                                         max_features = parmdict['Max Feats'],
                                         max_depth = parmdict['Max Depth']
                                        ) 
        clf.fit(Xtrain,ytrain)
        ascores.append(clf.score(Xtest, ytest))
        aoobs.append(clf.oob_score_)
    scores.append(ascores)
    oobs.append(aoobs)
        

16
1


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\ndimage\measurements.py:1359: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
17
1


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  >>> np.delete(arr, [1,3,5], None)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\ndimage\measurements.py:1359: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
3
1


...

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
10
1


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\ndimage\measurements.py:1359: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  >>> np.delete(arr, [1,3,5], None)


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
11
1


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  >>> np.delete(arr, [1,3,5], None)


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


In [226]:
data = scores
plt.figure()
plt.bar(range(len(data)), [np.mean(i) for i in data], yerr=[3*np.std(i) for i in data],width=0.3)
plt.hlines(1/3,0,8)
for i,a in enumerate(data):
    plt.scatter([i]*len(a), a, alpha=0.6, zorder=5)

In [330]:
# Single run RF on whole data matrix X. Repeat w shuffled labels many times. Compare best real alley OOB to dist of best nulls 

features = [
    'boundMaxs', #nb this is three separate features but they come as a set conceptually
    'auc',
    'com',
    'comval',
    'mean',
    'avgdrds',
    'maxdrds'
           ]

parmdict = {
    'nRuns':250, 
    'nTrees':25000, 
    'Max Depth':15, 
    'Max Feats':'auto',
    'Run type':'Repeated shuffle whole X RF',
    'Cell inclusion in population': '95pctile >=1Hz overall',
    'Visit inclusion in data matrix': 'Mean alley activity >=1 Hz'
    'Bootstrap: None',
    'Shuffle':True
    }

# run once get real score
realoobs = []
for alley in beltwayAlleys:
    print(alley)
    X,Y = setupAlleyData(alley, createVisitSummaryFeatures, features, bootstrap=False)
    clf = RandomForestClassifier(n_estimators=parmdict['nTrees'], 
                                     oob_score=True,
                                     max_features = parmdict['Max Feats'],
                                     max_depth = parmdict['Max Depth']
                                    ) 
    clf.fit(X,Y)
    realoobs.append(clf.oob_score_)
    
bestscore, bestalley = max(realoobs), np.argmax(realoobs)

# shuffle labels and compute best null alley score, store
allbestshuffs = []
for rep in range(parmdict['nRuns']):
    
    oobs = []
    for alley in beltwayAlleys:
        print(alley)
        X,Y = setupAlleyData(alley, createVisitSummaryFeatures, features, bootstrap=False)
        Y = np.random.permutation(Y) # return sorted label array
        clf = RandomForestClassifier(n_estimators=parmdict['nTrees'], 
                                         oob_score=True,
                                         max_features = parmdict['Max Feats'],
                                         max_depth = parmdict['Max Depth']
                                        ) 
        clf.fit(X,Y)
        oobs.append(clf.oob_score_)

    allbestshuffs.append(max(oobs))
    

        

16


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  >>> np.delete(arr, [1,3,5], None)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\ndimage\measurements.py:1359: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]


17


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  >>> np.delete(arr, [1,3,5], None)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\ndimage\measurements.py:1359: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]


3


...

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  >>> np.delete(arr, [1,3,5], None)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\ndimage\measurements.py:1359: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]


10


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  >>> np.delete(arr, [1,3,5], None)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:51: RuntimeWarning: All-NaN slice encountered


11


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  >>> np.delete(arr, [1,3,5], None)


## Using RF on a cell by cell basis to assess decodability of alley

In [8]:
def setupAlleyData_cell(unit, alley, representationFx, features):

    """
    Function that creates a single unit feature matrix for an alley
    along with a corresponding label vector
    
    Feature matrix X is (t,f) where t is number trials, f is number features 
    Label vector Y is (t,1) where each entry is the stimulus label for that trial
    Features f are found for a single cell via representrationFx which is the generic name
    for any fx that takes (Unit, visit) as args and returns a (f,1) array of the features
    for that cell for that visit. What the features are is determined by user in code of the rep fx.
    Matrix X is transformed via scikitlearn's preprocessing.StandardScalar().fit_transform() fx to have 0mean,unit var
    
    Returns X (t,f) array feature matrix trials by pop feature vector
            Y (t,1) array of trial labels

    """
    nvisits = len(unit.alleys[alley])
    X = []
    Y = []

    for visit in range(nvisits):

        stim = population[list(population.keys())[0]].alleys[alley][visit]['metadata']['stimulus']
        dataRow = np.empty((0))

        rm = representationFx(unit, alley, visit, features)
        X.append(rm)
        Y.append(stim)
                        
    X = np.asarray(X)
    Y = np.asarray(Y)
    X[np.where(~np.isfinite(X))] = 0
    X = preprocessing.StandardScaler().fit_transform(X)

    return X, Y

In [12]:
def parmString():
    string = ''
    for k,v in parmdict.items():
        string += f"{k}:{v}\n"
    for f in features:
        string +=f"{f}\n"
    return string

In [9]:
def runJackknife(X,Y):
    """
    Runs a Jackknife Random Forest. Remove each sample, compute OOB on remaining
    Return all oobs, same size as X
    """
    scores = []
    kf = KFold(n_splits=X.shape[0], shuffle=True)
    for train, test in kf.split(X):
        Xtrain,Xtest =X[train], X[test]
        Ytrain, Ytest = Y[train],Y[test]
        clf = RandomForestClassifier(n_estimators=parmdict['nTrees'], 
                                         oob_score=True,
                                         max_features = parmdict['Max Feats'],
                                         max_depth = parmdict['Max Depth']
                                        ) 
        clf.fit(Xtrain,Ytrain)
        scores.append(clf.oob_score_)
    return scores

In [10]:
features = [
    'rm'
           ]

parmdict = {
    'nRuns':'n', 
    'nTrees':2000, 
    'Max Depth':15, 
    'Max Feats':'auto',
    'Run type':'Single Cell Jackknife per alley',
    'Cell incl.': '95pctile >=1Hz overall',
    'Visit incl': '>=9 passes >=1Hzmax',
    'Resample':'Jackknife',
    'Shuffle':False
    }


cellResults = {name:{'oobs':[],'padj':None,'validAlleys':[]} for name in population.keys()}

for name, unit in population.items():
    print(name)
    validAlleys = []
    for i,alley in enumerate(Def.beltwayAlleys):
        print(alley)
        valid = Filt.checkMinimumPassesActivity(unit, alley, fr_thresh=1.0, pass_thresh=9)
        validAlleys.append(valid)
        if valid:
            print(f"Decoding {name}, alley {alley}")
            cellResults[name]['validAlleys'].append(i)
            X,Y = setupAlleyData_cell(unit, alley, createVisitSummaryFeatures, features)
            if parmdict['Shuffle'] == True:
                print('shuffle')
                Y = np.random.permutation(Y)
            oobs = runJackknife(X,Y)
            cellResults[name]['oobs'].append(oobs)
    if sum(validAlleys) > 0:
        cellResults[name]['padj'] = scipy.stats.norm.ppf(1-(0.05/sum(validAlleys))) # this padj value represents the factor you multiply the std to get the CI. Bonforonni correct based on usable alleys and convert to the ppf


TT1cl-maze1.1
16
17
3
1
Decoding TT1cl-maze1.1, alley 1
5
7
8
10
11
TT1cl-maze1.2
16
17
3
1
Decoding TT1cl-maze1.2, alley 1
5
Decoding TT1cl-maze1.2, alley 5
7
Decoding TT1cl-maze1.2, alley 7
8
Decoding TT1cl-maze1.2, alley 8
10
Decoding TT1cl-maze1.2, alley 10
11
Decoding TT1cl-maze1.2, alley 11
TT1cl-maze1.3
16
17
3
Decoding TT1cl-maze1.3, alley 3
1
5
7
8
10
Decoding TT1cl-maze1.3, alley 10
11
Decoding TT1cl-maze1.3, alley 11
TT12cl-maze1.1
16
17
3
1
5
7
8
10
11
TT12cl-maze1.3
16
Decoding TT12cl-maze1.3, alley 16
17
Decoding TT12cl-maze1.3, alley 17
3
Decoding TT12cl-maze1.3, alley 3
1
Decoding TT12cl-maze1.3, alley 1
5
Decoding TT12cl-maze1.3, alley 5
7
Decoding TT12cl-maze1.3, alley 7
8
Decoding TT12cl-maze1.3, alley 8
10
Decoding TT12cl-maze1.3, alley 10
11
Decoding TT12cl-maze1.3, alley 11
TT15cl-maze1.1
16
17
3
1
5
Decoding TT15cl-maze1.1, alley 5
7
8
10
11
TT15cl-maze1.2
16
Decoding TT15cl-maze1.2, alley 16
17
Decoding TT15cl-maze1.2, alley 17
3
1
5
Decoding TT15cl-maze1.2, all

In [13]:
string = parmString()
stamp = util.genTimestamp()
for name in list(population.keys())[:]:
    plt.figure(figsize=(6,6))
    plt.title(name)
    data = cellResults[name]['oobs']
    if data is not []:
        plt.bar(cellResults[name]['validAlleys'], [np.mean(i) for i in data], yerr=[cellResults[name]['padj']*np.std(i) for i in data],width=0.3)
        plt.hlines(1/3,0,8)
        for i,a in enumerate(data):
            plt.scatter([cellResults[name]['validAlleys'][i]]*len(a), a, alpha=0.6, zorder=5)
        plt.text(0.005, 0.2, string, fontsize=6, transform=plt.gcf().transFigure)
    plt.savefig(f'E:\Ratterdam\\{rat}\\random_forest\\single units\\{stamp}_{name}.png')
    plt.close()

### Reconciling single unit RF and single unit permutation tests

In [565]:
features = [
    'boundMaxs', #nb this is three separate features but they come as a set conceptually
    'auc',
    'com',
    'comval',
    'mean',
    'avgdrds',
    'maxdrds'
           ]

parmdict = {
    'nRuns':'n', 
    'nTrees':1000, 
    'Max Depth':15, 
    'Max Feats':'auto',
    'Run type':'Single Cell Jackknife per alley',
    'Cell inclusion in population': '95pctile >=1Hz overall',
    'Visit inclusion in data matrix': 'Mean alley activity >=1 Hz'
    'Bootstrap: None',
    'Shuffle':True
    }

unitName = "TT13cl-maze1.3"
unit = population[unitName]
alley = 7

X,Y = setupAlleyData_cell(unit, alley, createVisitSummaryFeatures, features)
Xa, Xb, Xc = X[np.where(Y=='A')[0],:], X[np.where(Y=='B')[0],:], X[np.where(Y=='C')[0],:]    

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  Parameters
